In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf
import pywt
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM, Dense, Activation
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
# Read CSV into a dataframe

# Setting the name of the Excel file which contains the solar generation data

file_name = 'PV_BY_Day.xlsx'

# Reading the Excel file using pandas. The engine 'openpyxl' is specified since it's a common engine for reading .xlsx files

df_generation = pd.read_excel(file_name, engine='openpyxl')

# Setting the name of the CSV file which contains the solar exposure data

file_name = 'IDCJAC0016_066207_1800_Data.csv'

# Reading the CSV file using pandas

df_exposure = pd.read_csv(file_name)

# Reading the 'totaldemand_nsw.csv' file into a dataframe

df_demand = pd.read_csv('totaldemand_nsw.csv', sep = ',', header = 0, parse_dates=['DATETIME'], dayfirst=True)

# Reading the 'temperature_nsw.csv' file into a dataframe

df_temp = pd.read_csv('temperature_nsw.csv', sep = ',', header = 0, parse_dates=['DATETIME'], dayfirst=True)

In [3]:
# Setting the name of the CSV files for the humidity data

file_name1 = 'humidity_wsp_20160916_20180729.xlsx'
file_name2 = 'humidity_wsp_20180730_20200610.xlsx'
file_name3 = 'humidity_wsp_20200611_20200930.xlsx'
file_name4 = 'humidity_wsp_20201001_20220731.xlsx'

# Reading the CSV files into dataframes

df_humidity_wsp1 = pd.read_excel(file_name1, engine='openpyxl')
df_humidity_wsp2 = pd.read_excel(file_name2, engine='openpyxl')
df_humidity_wsp3 = pd.read_excel(file_name3, engine='openpyxl')
df_humidity_wsp4 = pd.read_excel(file_name4, engine='openpyxl')

df_humidity_wsp = pd.concat([df_humidity_wsp1, df_humidity_wsp2, df_humidity_wsp3, df_humidity_wsp4], ignore_index=True)

In [4]:
# Define a list of strings which represent column names for a dataframe

column_names = ['date', 'time', 'wsp', 'humidity']

# Rename the columns of the 'df_humidity_wsp' dataframe

df_humidity_wsp.columns = column_names

In [5]:
# Convert 'date' column to datetime
df_humidity_wsp['date'] = pd.to_datetime(df_humidity_wsp['date'], format='%d/%m/%Y')
df_humidity_wsp['date'] = df_humidity_wsp['date'].dt.strftime('%Y-%m-%d')
                                         
# Group by 'date' and compute mean for 'wsp' and 'humidity'
agg_df_humidity_wsp = df_humidity_wsp.groupby('date').agg({'wsp': 'mean', 'humidity': 'mean'}).reset_index()

agg_df_humidity_wsp['date'] = pd.to_datetime(agg_df_humidity_wsp['date'], format='%Y-%m-%d')

In [6]:
#Filling in all nan humidity and wsp values with average for that date

agg_df_humidity_wsp['date'] = pd.to_datetime(agg_df_humidity_wsp['date'])

# Extract month and day from the date
agg_df_humidity_wsp['month'] = agg_df_humidity_wsp['date'].dt.month
agg_df_humidity_wsp['day'] = agg_df_humidity_wsp['date'].dt.day

# Calculate average values for wsp and humidity for each month-day combination
avg_values = agg_df_humidity_wsp.groupby(['month', 'day'])[['wsp', 'humidity']].mean().reset_index()

# Merge average values back to the original dataframe
agg_df_humidity_wsp = agg_df_humidity_wsp.merge(avg_values, on=['month', 'day'], suffixes=('', '_avg'))

# Replace NaN values with the average values
agg_df_humidity_wsp['wsp'].fillna(agg_df_humidity_wsp['wsp_avg'], inplace=True)
agg_df_humidity_wsp['humidity'].fillna(agg_df_humidity_wsp['humidity_avg'], inplace=True)

# Drop the auxiliary columns
agg_df_humidity_wsp.drop(columns=['month', 'day', 'wsp_avg', 'humidity_avg'], inplace=True)

In [7]:
# Extract the 'date' portion from the 'DATETIME' column of the 'df_demand' dataframe

df_demand['date'] = df_demand['DATETIME'].dt.date

# Convert the 'date' column in the 'df_demand' dataframe to a datetime format (from a date format)

df_demand['date'] = pd.to_datetime(df_demand['date'])

# Drop the first column (i.e., at index 0) of the 'df_demand' dataframe

df_demand = df_demand.drop(df_demand.iloc[:,0:1].columns, axis=1)

# Rename the 'TOTALDEMAND' column to 'total_demand' in the 'df_demand' dataframe

df_demand = df_demand.rename(columns={'TOTALDEMAND': 'total_demand'})

# Filter the 'df_demand' dataframe to only include rows where the 'date' is between 2016-09-16 and 2022-07-30 (inclusive)

df_demand = df_demand[df_demand['date'] < '2022-08-01']

df_demand = df_demand[df_demand['date'] >= '2016-09-16']

# Group the 'df_demand' dataframe by the 'date' column, summing the 'total_demand' for each date, and resetting the index

agg_df_demand = df_demand.groupby(['date'])['total_demand'].sum().reset_index()

In [8]:
# Extract the date portion from the 'DATETIME' column and store it in a new 'date' column.

df_temp['date'] = df_temp['DATETIME'].dt.date

# Convert the 'date' column from strings to datetime objects.

df_temp['date'] = pd.to_datetime(df_temp['date'])

# Drop the first column from the DataFrame

df_temp = df_temp.drop(df_temp.iloc[:,0:1].columns, axis=1)

# Rename the 'TEMPERATURE' column to 'temperature'.

df_temp = df_temp.rename(columns={'TEMPERATURE': 'temperature'})

# Filter the 'df_temp' dataframe to only include rows where the 'date' is between 2016-09-16 and 2022-07-30 (inclusive)

df_temp = df_temp[df_temp['date'] < '2022-08-01']
df_temp = df_temp[df_temp['date'] >= '2016-09-16']

# Group the DataFrame by 'date' and calculate the mean temperature for each date group.

agg_df_temp = df_temp.groupby(['date'])['temperature'].mean().reset_index()

In [9]:

# Group the DataFrame 'df_generation' by 'INTERVAL_DATETIME' and calculate the mean of 'GWh' for each group.

df_generation = df_generation.groupby('INTERVAL_DATETIME')['GWh'].mean().reset_index()

# Convert the 'INTERVAL_DATETIME' column to datetime objects by extracting only the date part.

df_generation['INTERVAL_DATETIME'] = pd.to_datetime(df_generation['INTERVAL_DATETIME'].dt.date)

# Define a list of column names.

column_names = ['date', 'solar_generation']

# Rename the columns of 'df_generation' using the defined column names.

df_generation.columns = column_names

# The date range for this project is 16/09/2016 - 01/08/2022. The next section filters the data to this scoped range.

df_generation = df_generation[df_generation['date'] < '2022-08-01']


# Group the filtered DataFrame by 'date' and calculate the sum of 'solar_generation' for each date group.

agg_df_generation = df_generation.groupby(['date'])['solar_generation'].sum().reset_index()

In [10]:
# Drop the first two columns from the DataFrame 'df_exposure'.

df_exposure = df_exposure.drop(df_exposure.columns[[0,1]], axis = 1)

# Define a list of column names.

column_names = ['year', 'month', 'day', 'daily_solar_exposure']

# Rename the columns of 'df_exposure' using the defined column names.

df_exposure.columns = column_names

# Create a new 'date' column by combining 'year', 'month', and 'day' columns into a datetime object.

df_exposure['date'] = pd.to_datetime(df_exposure[['year', 'month', 'day']])

# Filter the 'df_exposure' dataframe to only include rows where the 'date' is between 2016-09-16 and 2022-07-30 (inclusive)

df_exposure = df_exposure[df_exposure['date'] < '2022-08-01']
df_exposure = df_exposure[df_exposure['date'] >= '2016-09-16']

# There is one null daily_solar_exposure value on the 26th of November 2017. In order to fix this issue, we can use the average solar exposure for the 26th November.

avg_exposure = df_exposure[(df_exposure['month'] == 11) & (df_exposure['day'] == 26)]['daily_solar_exposure'].mean()
df_exposure.loc[(df_exposure['year'] == 2017) & (df_exposure['month'] == 11) & (df_exposure['day'] == 26), 'daily_solar_exposure'] = avg_exposure

# Dropping fields that are no longer required

df_exposure = df_exposure.drop(columns=['year', 'month', 'day'])

In [11]:
# Aggregating the Solar Generation Data to daily

agg_df_generation = df_generation.groupby(['date'])['solar_generation'].sum().reset_index()

In [12]:
# Joining datasets using outer joins on date

merged_df = agg_df_demand.merge(agg_df_temp, on='date', how='outer')\
              .merge(agg_df_generation, on='date', how='outer')\
              .merge(df_exposure, on='date', how='outer')\
              .merge(agg_df_humidity_wsp, on='date', how='outer')

In [13]:
# Only keep the columns 'date', 'total_demand', 'temperature', 'wsp', and 'humidity',

df = merged_df[['date', 'total_demand', 'temperature', 'wsp', 'humidity']]

# Add a new column 'day_of_year' to the 'df' DataFrame, which extracts the day of the year from the 'date' column.

df['day_of_year'] = df['date'].dt.dayofyear

<ipython-input-13-af73b417fd1d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_year'] = df['date'].dt.dayofyear


In [14]:
# Calculate a normalized 'day_of_year' value in the range [0, 2*pi].
df["day_of_year_norm"] = 2 * math.pi * df["day_of_year"] / df["day_of_year"].max()

# Calculate the cosine of the normalized 'day_of_year_norm' values and store them in a new column 'day_of_year_cos'.

df["day_of_year_cos"] = np.cos(df["day_of_year_norm"])

# Calculate the sine of the normalized 'day_of_year_norm' values and store them in a new column 'day_of_year_sin'.

df["day_of_year_sin"] = np.sin(df["day_of_year_norm"])

In [15]:
# Store the dates before removing the column
date_values = df['date'] 

# Only keep the columns 'total_demand', 'temperature', 'wsp','humidity', 'day_of_year_cos', 'day_of_year_sin'
df = df[['total_demand', 'temperature', 'wsp', 'humidity', 'day_of_year_cos', 'day_of_year_sin']]

In [16]:
# Setting the seed for reproducibility

seed_value = 5152
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [17]:
# Splitting data into train, validation, and test sets without shuffling to maintain time order

train_df, temp_df = train_test_split(df, test_size=0.4, shuffle=False)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)


In [18]:
# Normalization

# Create an instance of the MinMaxScaler

scaler = MinMaxScaler()

# Normalize the 'train_df' dataset and store the result in the 'train' variable.
# The 'fit_transform' method scales the data and also computes the scaling parameters based on the training data.

train = scaler.fit_transform(train_df)

# Normalize the 'valid_df' dataset using the scaling parameters learned from the training data.

valid = scaler.transform(valid_df)

# Normalize the 'test_df' dataset using the same scaling parameters learned from the training data.

test = scaler.transform(test_df)

In [19]:
# Function to create data sets suitable for LSTM input

def create_dataset(dataset, look_back=10):

    # Initialize empty lists to store sequences of input features (dataX) and target values (dataY).
    
    dataX, dataY = [], []
    
    # Iterate through the dataset to create sequences.
    for i in range(len(dataset)-look_back-1):
                
        # Extract a subsequence of 'look_back' time steps from the dataset.
        
        a = dataset[i:(i+look_back), :]
        
        # Append the subsequence to the 'dataX' list.        
        
        dataX.append(a)
        
        # Append the next time step's value (target) to the 'dataY' list.        
        
        dataY.append(dataset[i + look_back, 0])
        
    # Convert the lists of sequences into numpy arrays and return them.        
        
    return np.array(dataX), np.array(dataY)

In [20]:
# Function to apply wavelet transform to the data

def wavelet_transform(data, wavelet='db1'):
    
    # Perform a single-level discrete wavelet transform on the input data using the specified wavelet.
    coeffs = pywt.dwt(data, wavelet=wavelet)
    
    # Stack the approximation and detail coefficients horizontally to create the transformed data.
    transformed_data = np.hstack(coeffs)
    
    # Ensure output shape consistency:
    # If the transformed data has more columns than the input data, remove the last column.
    if transformed_data.shape[1] > data.shape[1]:
        transformed_data = transformed_data[:, :-1]
        
    # If the transformed data has fewer columns than the input data, pad with zeros.        
        
    elif transformed_data.shape[1] < data.shape[1]:
        padding_length = data.shape[1] - transformed_data.shape[1]
        padding = np.zeros((transformed_data.shape[0], padding_length))
        transformed_data = np.hstack((transformed_data, padding))

    # Return the transformed data.        
        
    return transformed_data

In [21]:
# Function to define and train LSTM model

def lstm_model(learning_rate, look_back, activation, batch_size, wavelet):
    
    # Convert batch_size and look_back to integers.
    
    batch_size = int(batch_size)
    look_back = int(look_back)
    
    # Create datasets with the specified look-back period.
    trainX, trainY = create_dataset(train, look_back)
    validX, validY = create_dataset(valid, look_back)
    
    # Apply wavelet transform if the wavelet parameter is greater than 0.5.
    if wavelet > 0.5:
        for i in range(trainX.shape[2]):
            trainX[:, :, i] = wavelet_transform(trainX[:, :, i])
            validX[:, :, i] = wavelet_transform(validX[:, :, i])
    
    # Define the architecture of the LSTM model.
    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Activation(activation))
    model.add(Dense(1))
    
    # Compile the model with the specified learning rate.
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    # Implement early stopping to prevent overfitting.
    early_stop = EarlyStopping(monitor='val_loss', patience=15)
    
    # Train the model on the training data.
    model.fit(trainX, trainY, epochs=1000, batch_size=batch_size, 
              validation_data=(validX, validY), callbacks=[early_stop], verbose=0)
    
    # Evaluate the model on the validation data and return the negative validation loss (to minimize).
    val_loss = model.evaluate(validX, validY, verbose=0)
    return -val_loss  # Return negative of validation loss to minimize

In [22]:
# Function to perform Bayesian optimization for LSTM model parameters

def optimize_lstm():
    
    # Wrapper function for Bayesian optimization
    
    def lstm_crossval(learning_rate, look_back, activation, batch_size, wavelet):

        # Map the activation index to the corresponding activation function.
        
        activations = {0: 'gelu', 1: 'relu', 2: 'tanh'}
        activation_idx = int(np.round(activation))
        
        # Call the lstm_model function with the specified hyperparameters.        
        
        return lstm_model(learning_rate, look_back, activations[activation_idx], batch_size, wavelet)
    
    # Setting up the Bayesian optimizer
    optimizer = BayesianOptimization(
        f=lstm_crossval,
        pbounds={
            "learning_rate": (1e-5, 1e-2),
            "look_back": (10, 70),
            "activation": (0, 2),
            "batch_size": (1, 128),
            "wavelet": (0, 1)
        },
        random_state=42,
        verbose=2
    )
    
    # Running the optimization
    optimizer.maximize(init_points=10, n_iter=1000)

    # Printing the best results
    print("Final result:", optimizer.max)


In [23]:
# Run the optimization

optimize_lstm()

|   iter    |  target   | activa... | batch_... | learni... | look_back |  wavelet  |
-------------------------------------------------------------------------------------
| 1         | -0.009309 | 0.7491    | 121.7     | 0.007323  | 45.92     | 0.156     |
| 2         | -0.01178  | 0.312     | 8.377     | 0.008663  | 46.07     | 0.7081    |
| 3         | -0.008967 | 0.04117   | 124.2     | 0.008326  | 22.74     | 0.1818    |
| 4         | -0.008443 | 0.3668    | 39.64     | 0.005252  | 35.92     | 0.2912    |
| 5         | -0.00733  | 1.224     | 18.72     | 0.002929  | 31.98     | 0.4561    |
| 6         | -0.007427 | 1.57      | 26.36     | 0.005147  | 45.54     | 0.04645   |
| 7         | -0.01424  | 1.215     | 22.66     | 0.0006599 | 66.93     | 0.9656    |
| 8         | -0.01012  | 1.617     | 39.69     | 0.0009857 | 51.05     | 0.4402    |
| 9         | -0.01232  | 0.2441    | 63.89     | 0.0003535 | 64.56     | 0.2588    |
| 10        | -0.007955 | 1.325     | 40.59     | 0.00

| 58        | -0.01328  | 1.415     | 18.36     | 0.0007089 | 31.43     | 0.8757    |
| 59        | -0.01211  | 1.131     | 31.36     | 0.001084  | 60.78     | 0.955     |
| 60        | -0.009091 | 0.7084    | 17.56     | 0.002276  | 31.44     | 0.4596    |
| 61        | -0.01938  | 0.7956    | 110.0     | 0.009558  | 52.66     | 0.5958    |
| 62        | -0.008376 | 1.469     | 33.97     | 0.008328  | 61.48     | 0.6333    |
| 63        | -0.01055  | 1.355     | 79.1      | 0.008315  | 62.05     | 0.9796    |
| 64        | -0.01108  | 0.3777    | 114.5     | 0.006621  | 40.87     | 0.7274    |
| 65        | -0.00669  | 0.3148    | 33.41     | 0.003508  | 22.39     | 0.2575    |
| 66        | -0.01051  | 1.462     | 30.97     | 0.003611  | 18.18     | 0.6698    |
| 67        | -0.009383 | 1.686     | 36.87     | 0.001474  | 22.1      | 0.05968   |
| 68        | -0.007121 | 0.1554    | 31.3      | 0.004462  | 18.58     | 0.6869    |
| 69        | -0.009179 | 0.6526    | 1.72      | 0.00

| 116       | -0.015    | 0.6299    | 43.72     | 0.0002473 | 66.95     | 0.9237    |
| 117       | -0.01198  | 0.8752    | 117.2     | 0.00987   | 39.09     | 0.1707    |
| 118       | -0.00845  | 0.2082    | 119.3     | 0.004233  | 34.53     | 0.7636    |
| 119       | -0.009346 | 1.41      | 120.9     | 0.009661  | 67.75     | 0.3062    |
| 120       | -0.00884  | 1.49      | 47.35     | 0.007927  | 53.01     | 0.2347    |
| 121       | -0.01291  | 1.177     | 31.98     | 8.04e-05  | 18.83     | 0.7519    |
| 122       | -0.01429  | 1.776     | 72.76     | 5.86e-05  | 49.54     | 0.6503    |
| 123       | -0.01568  | 1.302     | 105.4     | 0.004378  | 61.91     | 0.7735    |
| 124       | -0.00814  | 0.3247    | 78.66     | 0.0039    | 14.12     | 0.1643    |
| 125       | -0.01395  | 1.346     | 125.8     | 0.0006968 | 23.08     | 0.6713    |
| 126       | -0.01125  | 1.057     | 80.36     | 0.003493  | 65.83     | 0.4886    |
| 127       | -0.008409 | 0.7942    | 85.54     | 0.00

| 174       | -0.011    | 1.586     | 83.19     | 0.00257   | 68.63     | 0.2821    |
| 175       | -0.009001 | 0.7001    | 104.4     | 0.005308  | 44.74     | 0.01762   |
| 176       | -0.008259 | 1.272     | 117.5     | 0.008644  | 14.99     | 0.03276   |
| 177       | -0.01544  | 1.405     | 22.79     | 0.0003052 | 60.56     | 0.645     |
| 178       | -0.01064  | 0.5591    | 99.76     | 0.005419  | 26.27     | 0.839     |
| 179       | -0.007902 | 0.02125   | 17.8      | 0.008086  | 18.75     | 0.9712    |
| 180       | -0.01014  | 1.979     | 75.91     | 0.009307  | 14.73     | 0.4372    |
| 181       | -0.009468 | 1.029     | 85.78     | 0.003832  | 36.77     | 0.08534   |
| 182       | -0.007731 | 0.7713    | 4.23      | 0.000803  | 16.21     | 0.4215    |
| 183       | -0.01001  | 1.614     | 108.8     | 0.008645  | 52.37     | 0.9891    |
| 184       | -0.009621 | 1.6       | 125.9     | 0.002272  | 23.35     | 0.2027    |
| 185       | -0.009621 | 0.2667    | 17.22     | 0.00

| 232       | -0.01139  | 0.4144    | 26.03     | 0.0006747 | 65.87     | 0.2653    |
| 233       | -0.01329  | 0.4696    | 124.2     | 0.004176  | 31.49     | 0.629     |
| 234       | -0.006879 | 0.2474    | 47.77     | 0.005667  | 20.44     | 0.7991    |
| 235       | -0.01451  | 0.9569    | 51.48     | 0.003929  | 11.03     | 0.657     |
| 236       | -0.007167 | 0.5708    | 32.98     | 0.007524  | 22.75     | 0.4237    |
| 237       | -0.01008  | 1.377     | 53.08     | 0.002911  | 34.38     | 0.8223    |
| 238       | -0.008763 | 0.118     | 33.19     | 0.008337  | 22.51     | 0.03243   |
| 239       | -0.00677  | 0.7585    | 102.1     | 0.003823  | 33.54     | 0.2343    |
| 240       | -0.01502  | 0.5494    | 19.31     | 0.0001933 | 49.02     | 0.9895    |
| 241       | -0.01404  | 1.471     | 50.17     | 0.001235  | 39.08     | 0.9152    |
| 242       | -0.01472  | 0.6518    | 115.3     | 9.349e-05 | 63.07     | 0.7682    |
| 243       | -0.007902 | 0.4888    | 86.61     | 0.00

...

| 754       | -0.00967  | 1.609     | 115.4     | 0.002653  | 37.93     | 0.776     |
| 755       | -0.006193 | 0.9312    | 3.562     | 0.008008  | 31.52     | 0.2693    |
| 756       | -0.007549 | 1.189     | 18.47     | 0.006043  | 33.52     | 0.2974    |
| 757       | -0.01332  | 1.345     | 113.9     | 0.0005827 | 23.9      | 0.8254    |
| 758       | -0.01052  | 1.737     | 38.43     | 0.006125  | 17.0      | 0.7652    |
| 759       | -0.01058  | 1.522     | 81.85     | 0.008747  | 42.13     | 0.6872    |
| 760       | -0.008333 | 0.6933    | 78.9      | 0.007843  | 60.25     | 0.4603    |
| 761       | -0.009088 | 0.2416    | 108.8     | 0.004885  | 28.38     | 0.1642    |
| 762       | -0.006718 | 1.89      | 8.006     | 0.005895  | 22.42     | 0.04388   |
| 763       | -0.009336 | 1.198     | 4.2       | 0.008939  | 31.26     | 0.06982   |
| 764       | -0.01048  | 0.2296    | 98.4      | 0.003667  | 12.0      | 0.6584    |
| 765       | -0.009865 | 1.104     | 80.44     | 0.00

| 812       | -0.009783 | 1.295     | 51.58     | 0.001185  | 58.07     | 0.8443    |
| 813       | -0.01564  | 0.176     | 80.2      | 0.005132  | 66.64     | 0.9734    |
| 814       | -0.007331 | 0.1377    | 39.49     | 0.008528  | 29.98     | 0.1402    |
| 815       | -0.008925 | 1.674     | 43.56     | 0.004167  | 27.7      | 0.6793    |
| 816       | -0.007942 | 0.2024    | 124.0     | 0.003919  | 36.72     | 0.3255    |
| 817       | -0.009541 | 0.002542  | 112.8     | 0.00331   | 34.12     | 0.6921    |
| 818       | -0.007958 | 0.7934    | 19.89     | 0.001968  | 32.94     | 0.6072    |
| 819       | -0.00839  | 1.494     | 122.4     | 0.005826  | 55.07     | 0.8494    |
| 820       | -0.007153 | 1.333     | 4.596     | 0.005662  | 31.71     | 0.2566    |
| 821       | -0.008254 | 0.4365    | 100.9     | 0.002024  | 22.76     | 0.2734    |
| 822       | -0.007505 | 1.713     | 1.037     | 0.009921  | 39.2      | 0.5417    |
| 823       | -0.006321 | 1.085     | 3.675     | 0.00

| 870       | -0.01084  | 0.5233    | 117.0     | 0.004941  | 46.85     | 0.7215    |
| 871       | -0.01022  | 1.073     | 43.93     | 0.002056  | 69.18     | 0.1062    |
| 872       | -0.01559  | 0.9319    | 115.9     | 0.002572  | 53.59     | 0.6638    |
| 873       | -0.01045  | 1.565     | 70.78     | 0.002545  | 17.22     | 0.8694    |
| 874       | -0.00695  | 1.314     | 73.27     | 0.004686  | 33.75     | 0.2922    |
| 875       | -0.01004  | 0.5191    | 116.6     | 0.007348  | 50.22     | 0.2904    |
| 876       | -0.00833  | 1.789     | 4.344     | 0.009053  | 31.16     | 0.3632    |
| 877       | -0.0087   | 1.427     | 50.97     | 0.006935  | 25.05     | 0.5494    |
| 878       | -0.01103  | 1.808     | 36.61     | 0.003193  | 62.38     | 0.9373    |
| 879       | -0.006578 | 0.9759    | 22.0      | 0.004141  | 28.77     | 0.1454    |
| 880       | -0.008859 | 1.861     | 64.88     | 0.001853  | 28.28     | 0.4316    |
| 881       | -0.008607 | 1.048     | 70.72     | 0.00

| 928       | -0.006937 | 1.114     | 4.391     | 0.001306  | 31.91     | 0.3759    |
| 929       | -0.00962  | 1.839     | 4.083     | 0.0008815 | 31.67     | 0.6666    |
| 930       | -0.008235 | 1.404     | 3.903     | 0.006865  | 31.5      | 0.7117    |
| 931       | -0.0076   | 1.236     | 10.41     | 0.003297  | 48.07     | 0.4642    |
| 932       | -0.00726  | 1.548     | 42.51     | 0.008362  | 38.17     | 0.1789    |
| 933       | -0.01063  | 1.834     | 8.119     | 0.001433  | 22.05     | 0.9186    |
| 934       | -0.007942 | 0.0676    | 29.59     | 0.0034    | 18.34     | 0.3866    |
| 935       | -0.008963 | 0.9446    | 3.315     | 0.0078    | 31.19     | 0.345     |
| 936       | -0.009018 | 0.3048    | 78.23     | 0.008536  | 50.71     | 0.4191    |
| 937       | -0.009967 | 0.5066    | 52.03     | 0.008586  | 29.93     | 0.9908    |
| 938       | -0.00883  | 0.3957    | 92.87     | 0.003759  | 20.72     | 0.6655    |
| 939       | -0.01243  | 0.9429    | 15.25     | 0.00

| 986       | -0.01069  | 0.1048    | 46.22     | 0.002771  | 17.81     | 0.5476    |
| 987       | -0.007437 | 1.962     | 24.96     | 0.00483   | 39.19     | 0.1606    |
| 988       | -0.007386 | 0.6815    | 74.36     | 0.008267  | 11.04     | 0.7376    |
| 989       | -0.007436 | 1.181     | 16.38     | 0.008152  | 10.59     | 0.9513    |
| 990       | -0.006599 | 1.115     | 3.731     | 0.008104  | 31.52     | 0.3748    |
| 991       | -0.009001 | 1.842     | 93.07     | 0.00812   | 49.87     | 0.03233   |
| 992       | -0.008241 | 0.1413    | 59.63     | 0.004603  | 23.28     | 0.8382    |
| 993       | -0.007597 | 1.795     | 8.076     | 0.003769  | 22.53     | 0.558     |
| 994       | -0.01007  | 1.68      | 27.78     | 0.001645  | 24.82     | 0.2571    |
| 995       | -0.007124 | 0.3662    | 31.45     | 0.002665  | 48.68     | 0.4645    |
| 996       | -0.01302  | 0.6899    | 106.4     | 0.003621  | 53.38     | 0.9728    |
| 997       | -0.009734 | 1.179     | 3.694     | 0.00